In [8]:
%load_ext autoreload
%autoreload 2

import duckdb
import json
import os
import pandas as pd
import pathlib

from ibge import *
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# Initialize DuckDB connection
conn = duckdb.connect("piper.db")
local_file = "gbif"
!ls gbif/

Piper+0.json	 Piper+1943.json  Piper+2886.json  Piper+3828.json
Piper+1000.json  Piper+1944.json  Piper+2887.json  Piper+3829.json
Piper+1001.json  Piper+1945.json  Piper+2888.json  Piper+382.json
Piper+1002.json  Piper+1946.json  Piper+2889.json  Piper+3830.json
Piper+1003.json  Piper+1947.json  Piper+288.json   Piper+3831.json
Piper+1004.json  Piper+1948.json  Piper+2890.json  Piper+3832.json
Piper+1005.json  Piper+1949.json  Piper+2891.json  Piper+3833.json
Piper+1006.json  Piper+194.json   Piper+2892.json  Piper+3834.json
Piper+1007.json  Piper+1950.json  Piper+2893.json  Piper+3835.json
Piper+1008.json  Piper+1951.json  Piper+2894.json  Piper+3836.json
Piper+1009.json  Piper+1952.json  Piper+2895.json  Piper+3837.json
Piper+100.json	 Piper+1953.json  Piper+2896.json  Piper+3838.json
Piper+1010.json  Piper+1954.json  Piper+2897.json  Piper+3839.json
Piper+1011.json  Piper+1955.json  Piper+2898.json  Piper+383.json
Piper+1012.json  Piper+1956.json  Piper+2899.json  Piper+3840.json
P

## GBIF
### Adiciono os registros no duckdb (via parquet)

In [10]:
conn.execute("DROP TABLE IF EXISTS gbif")
conn.execute(f"""
        CREATE TABLE gbif AS
        SELECT * FROM read_parquet('{local_file}', union_by_name=True)
    """)

result = conn.execute("SELECT COUNT(*) FROM gbif").fetchone()
print(f"Registros carregados na tabela: {result[0]:,}")

Registros carregados na tabela: 44,800


In [11]:
conn.execute("DESCRIBE gbif").df()

,column_name,column_type,null,key,default,extra
0,key,BIGINT,YES,None,None,None
1,datasetKey,VARCHAR,YES,None,None,None
2,publishingOrgKey,VARCHAR,YES,None,None,None
3,installationKey,VARCHAR,YES,None,None,None
4,hostingOrganizationKey,VARCHAR,YES,None,None,None
...,...,...,...,...,...,...
291,organismScope,VARCHAR,YES,None,None,None
292,locationAccordingTo,VARCHAR,YES,None,None,None
293,verbatimDepth,VARCHAR,YES,None,None,None
294,extensions.http://purl.org/germplasm/germplasm...,"STRUCT(""http://purl.org/germplasm/germplasmTer...",YES,None,None,None


In [12]:
conn.execute("SELECT * FROM gbif LIMIT 5").df()

,key,datasetKey,publishingOrgKey,installationKey,hostingOrganizationKey,publishingCountry,protocol,lastCrawled,lastParsed,crawlId,...,extensions.http://rs.gbif.org/terms/1.0/Image,lithostratigraphicTerms,nameAccordingToID,vitality,extensions.http://data.ggbn.org/schemas/ggbn/terms/Preparation,organismScope,locationAccordingTo,verbatimDepth,extensions.http://purl.org/germplasm/germplasmTerm#GermplasmAccession,http://unknown.org/typifiedName
0,3346478634,d8cd16ba-bb74-4420-821e-083f2bac17c2,ada9d123-ddb4-467d-8891-806ea8d94230,17a83780-3060-4851-9d6f-029d5fcb81c9,fbca90e3-8aed-48b1-84e3-369afbd000ce,GB,EML,2025-11-15T12:04:43.857+00:00,2025-11-15T12:57:08.044+00:00,248,...,<NA>,None,None,None,<NA>,None,None,None,<NA>,None
1,3349699357,d8cd16ba-bb74-4420-821e-083f2bac17c2,ada9d123-ddb4-467d-8891-806ea8d94230,17a83780-3060-4851-9d6f-029d5fcb81c9,fbca90e3-8aed-48b1-84e3-369afbd000ce,GB,EML,2025-11-15T12:04:43.857+00:00,2025-11-15T12:57:07.586+00:00,248,...,<NA>,None,None,None,<NA>,None,None,None,<NA>,None
2,3349843297,d8cd16ba-bb74-4420-821e-083f2bac17c2,ada9d123-ddb4-467d-8891-806ea8d94230,17a83780-3060-4851-9d6f-029d5fcb81c9,fbca90e3-8aed-48b1-84e3-369afbd000ce,GB,EML,2025-11-15T12:04:43.857+00:00,2025-11-15T12:58:00.495+00:00,248,...,<NA>,None,None,None,<NA>,None,None,None,<NA>,None
3,3346279664,d8cd16ba-bb74-4420-821e-083f2bac17c2,ada9d123-ddb4-467d-8891-806ea8d94230,17a83780-3060-4851-9d6f-029d5fcb81c9,fbca90e3-8aed-48b1-84e3-369afbd000ce,GB,EML,2025-11-15T12:04:43.857+00:00,2025-11-15T12:56:17.402+00:00,248,...,<NA>,None,None,None,<NA>,None,None,None,<NA>,None
4,3349718108,d8cd16ba-bb74-4420-821e-083f2bac17c2,ada9d123-ddb4-467d-8891-806ea8d94230,17a83780-3060-4851-9d6f-029d5fcb81c9,fbca90e3-8aed-48b1-84e3-369afbd000ce,GB,EML,2025-11-15T12:04:43.857+00:00,2025-11-15T12:57:07.505+00:00,248,...,<NA>,None,None,None,<NA>,None,None,None,<NA>,None


In [14]:
conn.execute("SELECT * FROM gbif").df()

,key,datasetKey,publishingOrgKey,installationKey,hostingOrganizationKey,publishingCountry,protocol,lastCrawled,lastParsed,crawlId,...,extensions.http://rs.gbif.org/terms/1.0/Image,lithostratigraphicTerms,nameAccordingToID,vitality,extensions.http://data.ggbn.org/schemas/ggbn/terms/Preparation,organismScope,locationAccordingTo,verbatimDepth,extensions.http://purl.org/germplasm/germplasmTerm#GermplasmAccession,http://unknown.org/typifiedName
0,3346478634,d8cd16ba-bb74-4420-821e-083f2bac17c2,ada9d123-ddb4-467d-8891-806ea8d94230,17a83780-3060-4851-9d6f-029d5fcb81c9,fbca90e3-8aed-48b1-84e3-369afbd000ce,GB,EML,2025-11-15T12:04:43.857+00:00,2025-11-15T12:57:08.044+00:00,248,...,<NA>,None,None,None,<NA>,None,None,None,<NA>,None
1,3349699357,d8cd16ba-bb74-4420-821e-083f2bac17c2,ada9d123-ddb4-467d-8891-806ea8d94230,17a83780-3060-4851-9d6f-029d5fcb81c9,fbca90e3-8aed-48b1-84e3-369afbd000ce,GB,EML,2025-11-15T12:04:43.857+00:00,2025-11-15T12:57:07.586+00:00,248,...,<NA>,None,None,None,<NA>,None,None,None,<NA>,None
2,3349843297,d8cd16ba-bb74-4420-821e-083f2bac17c2,ada9d123-ddb4-467d-8891-806ea8d94230,17a83780-3060-4851-9d6f-029d5fcb81c9,fbca90e3-8aed-48b1-84e3-369afbd000ce,GB,EML,2025-11-15T12:04:43.857+00:00,2025-11-15T12:58:00.495+00:00,248,...,<NA>,None,None,None,<NA>,None,None,None,<NA>,None
3,3346279664,d8cd16ba-bb74-4420-821e-083f2bac17c2,ada9d123-ddb4-467d-8891-806ea8d94230,17a83780-3060-4851-9d6f-029d5fcb81c9,fbca90e3-8aed-48b1-84e3-369afbd000ce,GB,EML,2025-11-15T12:04:43.857+00:00,2025-11-15T12:56:17.402+00:00,248,...,<NA>,None,None,None,<NA>,None,None,None,<NA>,None
4,3349718108,d8cd16ba-bb74-4420-821e-083f2bac17c2,ada9d123-ddb4-467d-8891-806ea8d94230,17a83780-3060-4851-9d6f-029d5fcb81c9,fbca90e3-8aed-48b1-84e3-369afbd000ce,GB,EML,2025-11-15T12:04:43.857+00:00,2025-11-15T12:57:07.505+00:00,248,...,<NA>,None,None,None,<NA>,None,None,None,<NA>,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44795,3351659529,d8cd16ba-bb74-4420-821e-083f2bac17c2,ada9d123-ddb4-467d-8891-806ea8d94230,17a83780-3060-4851-9d6f-029d5fcb81c9,fbca90e3-8aed-48b1-84e3-369afbd000ce,GB,EML,2025-11-15T12:04:43.857+00:00,2025-11-15T12:58:16.463+00:00,248,...,<NA>,None,None,None,<NA>,None,None,None,<NA>,None
44796,3349015149,d8cd16ba-bb74-4420-821e-083f2bac17c2,ada9d123-ddb4-467d-8891-806ea8d94230,17a83780-3060-4851-9d6f-029d5fcb81c9,fbca90e3-8aed-48b1-84e3-369afbd000ce,GB,EML,2025-11-15T12:04:43.857+00:00,2025-11-15T12:56:40.380+00:00,248,...,<NA>,None,None,None,<NA>,None,None,None,<NA>,None
44797,3818360720,d8cd16ba-bb74-4420-821e-083f2bac17c2,ada9d123-ddb4-467d-8891-806ea8d94230,17a83780-3060-4851-9d6f-029d5fcb81c9,fbca90e3-8aed-48b1-84e3-369afbd000ce,GB,EML,2025-11-15T12:04:43.857+00:00,2025-11-15T12:56:58.845+00:00,248,...,<NA>,None,None,None,<NA>,None,None,None,<NA>,None
44798,3346994240,d8cd16ba-bb74-4420-821e-083f2bac17c2,ada9d123-ddb4-467d-8891-806ea8d94230,17a83780-3060-4851-9d6f-029d5fcb81c9,fbca90e3-8aed-48b1-84e3-369afbd000ce,GB,EML,2025-11-15T12:04:43.857+00:00,2025-11-15T12:56:25.125+00:00,248,...,<NA>,None,None,None,<NA>,None,None,None,<NA>,None
